# 📊 Exploratory Data Analysis (EDA)

Khám phá và phân tích dữ liệu FreshRetail dataset.

## Mục tiêu
- Hiểu cấu trúc dữ liệu
- Phát hiện patterns và anomalies
- Kiểm tra data quality
- Visualize distributions và trends


In [1]:
# Setup
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Add project root to path
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.config import setup_project_path, get_dataset_config, setup_logging
from src.pipelines._01_load_data import load_data
from src.utils.data_quality import DataQualityMonitor
from src.utils.validation import comprehensive_validation

setup_project_path()
setup_logging()

# Import display for Jupyter notebooks
try:
    from IPython.display import display
except ImportError:
    # Fallback if not in Jupyter
    display = print

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


Added D:\datastorm\E-Grocery_Forecaster\src to sys.path
Data quality libraries not installed. Install with: pip install great-expectations evidently


ModuleNotFoundError: No module named 'great_expectations'

## 1. Load Data và Overview


In [ ]:
# Load data
dataframes, config = get_dataset_config(), None
dataframes, config = load_data()

print("=" * 70)
print("DATASET OVERVIEW")
print("=" * 70)
print(f"Dataset: {config['name']}")
print(f"Temporal Unit: {config['temporal_unit']}")
print(f"Time Column: {config['time_column']}")
print(f"Target Column: {config['target_column']}")
print(f"\nLoaded DataFrames: {len(dataframes)}")
print("-" * 70)

for name, df in dataframes.items():
    print(f"\n{name.upper()}:")
    print(f"  Shape: {df.shape}")
    print(f"  Columns: {list(df.columns)[:10]}...")
    print(f"  Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"  Missing values: {df.isnull().sum().sum()}")
    if len(df) > 0:
        print(f"  Date range: {df[config['time_column']].min()} to {df[config['time_column']].max()}")


## 2. Data Quality Check


In [ ]:
# Check data quality for each dataframe
quality_monitor = DataQualityMonitor()

for name, df in dataframes.items():
    print(f"\n{'='*70}")
    print(f"DATA QUALITY CHECK: {name.upper()}")
    print(f"{'='*70}")
    
    validation_results = comprehensive_validation(df, verbose=True)
    quality_score = validation_results.get('quality_score', 0)
    
    print(f"\nOverall Quality Score: {quality_score}/100")
    
    if quality_score >= 90:
        print("✅ EXCELLENT")
    elif quality_score >= 75:
        print("⚠️ GOOD")
    else:
        print("❌ POOR - Needs attention")
    
    # Store results
    quality_monitor.store_validation_results(name, validation_results)


## 3. Sales Data Analysis


In [ ]:
# Get sales dataframe
sales_df = dataframes.get('sales') or dataframes.get('freshretail_train')
if sales_df is None:
    print("Sales dataframe not found!")
else:
    print(f"Sales Data Shape: {sales_df.shape}")
    print(f"\nFirst few rows:")
    display(sales_df.head())
    
    print(f"\nBasic Statistics:")
    display(sales_df.describe())
    
    print(f"\nData Types:")
    display(sales_df.dtypes)
    
    # Check for missing values
    print(f"\nMissing Values:")
    missing = sales_df.isnull().sum()
    display(missing[missing > 0])


## 4. Time Series Visualization


In [ ]:
# Aggregate sales by time period
if sales_df is not None and config:
    time_col = config['time_column']
    target_col = config['target_column']
    
    # Aggregate
    if config['temporal_unit'] == 'hour':
        # For hourly data, aggregate to daily
        sales_df[time_col] = pd.to_datetime(sales_df[time_col])
        sales_df['date'] = sales_df[time_col].dt.date
        ts_data = sales_df.groupby('date')[target_col].sum().reset_index()
        ts_data['date'] = pd.to_datetime(ts_data['date'])
    else:
        # For weekly data
        ts_data = sales_df.groupby(time_col)[target_col].sum().reset_index()
        ts_data = ts_data.sort_values(time_col)
    
    # Plot
    fig = px.line(ts_data, x=ts_data.columns[0], y=target_col, 
                  title='Sales Over Time',
                  labels={ts_data.columns[0]: 'Time', target_col: 'Sales'})
    fig.update_layout(height=500)
    fig.show()
    
    # Distribution
    fig2 = px.histogram(sales_df, x=target_col, 
                       title='Sales Distribution',
                       nbins=50)
    fig2.update_layout(height=400)
    fig2.show()
